In [ ]:
import ollama

## Intro

### Model Info

In [12]:
def list_models(ignore_tag=True):
    model_names = []
    for m in [m['name'] for m in ollama.list()['models']]:
        model_names.append(m.split(":")[0] if (ignore_tag and ":" in m) else m)
    return model_names

In [26]:
print(list_models())

['codellama', 'deepseek-llm', 'dolphin-phi', 'llama2', 'llama2-chinese', 'llama2-uncensored', 'llava', 'magicoder', 'meditron', 'mistral', 'mistral-openorca', 'neural-chat', 'nomic-embed-text', 'openchat', 'orca-mini', 'phi', 'sqlcoder', 'starling-lm', 'vicuna', 'wizard-math', 'yi']


In [20]:
model_dic = ollama.show('llama2')

In [21]:
model_dic.keys()

dict_keys(['license', 'modelfile', 'parameters', 'template', 'details'])

In [22]:
model_dic['modelfile']

'# Modelfile generated by "ollama show"\n# To build a new Modelfile based on this one, replace the FROM line with:\n# FROM llama2:latest\n\nFROM /usr/share/ollama/.ollama/models/blobs/sha256:8934d96d3f08982e95922b2b7a2c626a1fe873d7c3b06e8e56d7bc0a1fef9246\nTEMPLATE """[INST] <<SYS>>{{ .System }}<</SYS>>\n\n{{ .Prompt }} [/INST]\n"""\nPARAMETER stop "[INST]"\nPARAMETER stop "[/INST]"\nPARAMETER stop "<<SYS>>"\nPARAMETER stop "<</SYS>>"'

In [23]:
model_dic['parameters']

'stop                           "[INST]"\nstop                           "[/INST]"\nstop                           "<<SYS>>"\nstop                           "<</SYS>>"'

In [24]:
model_dic['template']

'[INST] <<SYS>>{{ .System }}<</SYS>>\n\n{{ .Prompt }} [/INST]\n'

In [25]:
model_dic['details']

{'parent_model': '',
 'format': 'gguf',
 'family': 'llama',
 'families': ['llama'],
 'parameter_size': '7B',
 'quantization_level': 'Q4_0'}

### Basic chat

In [4]:
model_name = 'llama2'

In [10]:
for model_name in ['llama2', 'mistral','phi','yi']:
    response = ollama.chat(model=model_name, messages=[
      {
        'role': 'user',
        'content': 'Why is the sky blue?',
      },
    ])
    answer = response['message']['content']
    print(f"Model: {model_name}\n\Answer: {answer}")

model: llama2
	answer:	 
The sky appears blue because of a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths. This is known as Rayleigh scattering.

As a result of this scattering, the blue light is dispersed throughout the atmosphere, giving the sky its blue appearance. The blue light is not just scattered in all directions, but it also travels further into the atmosphere than other colors, which means that the farther away from the horizon you look, the bluer the sky appears.

Other factors can also affect the color of the sky, such as dust and pollution, which can absorb certain wavelengths of light and make the sky appear more hazy or gray. However, under normal conditions, Rayleigh scattering is the main cause of the blue color of the sky.
model: mistral
	

In [15]:
response

{'model': 'yi',
 'created_at': '2024-03-24T15:07:20.116695343Z',
 'message': {'role': 'assistant',
  'content': "The color of the sky depends on several factors, including the scattering of sunlight by air molecules and aerosols (fine particles in the atmosphere). Here are some reasons why the sky appears blue:\n\n1. Rayleigh Scattering: When sunlight travels through the atmosphere, it interacts with atoms and molecules within it. The most abundant molecule is nitrogen, but there are also oxygen, argon, water vapor, dust, and other pollutants present. These particles scatter light from the Sun in all directions. The shorter wavelengths of blue light are more easily scattered than the longer ones (e.g., red) because they travel through a smaller volume of air. This scattering of blue light is known as Rayleigh scattering after Lord Rayleigh who first described it for this application.\n\n2. Albedo: Snow and ice have an albedo that is much higher than the albedo of clouds. The snow-cover

### Streaming chat

In [16]:
model_name = "yi"
stream = ollama.chat(
    model=model_name,
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

The color of the sky depends on many factors, but one of the most significant reasons why it appears blue is because of the Earth's atmosphere. When sunlight travels through our atmosphere, it interacts with particles and gases in the air to scatter certain colors more than others. The blue wavelengths of light are scattered the most due to their shorter wavelength (blue has a higher energy than red). This scattering of blue wavelengths causes us to see blue sky.

To understand this phenomenon better, let's consider what happens when sunlight reaches our atmosphere. When blue light tries to pass through the atmosphere, it encounters tiny particles such as dust and water vapor that are present in varying amounts at different altitudes. These particles act like little prisms, bending or diffracting the various colors of white light into their constituent wavelengths (color separation). The shorter wavelength blue light gets scattered more effectively by these particles than the longer re

### Generate

In [ ]:
model_name = 'llama2'
prompt = 'Why is the sky blue?'
resp = ollama.generate(model=model_name, prompt=prompt)

In [18]:
print(resp['response'])


The sky appears blue because of a phenomenon called Rayleigh scattering, which occurs when sunlight travels through the Earth's atmosphere. The atmosphere contains tiny molecules of gases such as nitrogen and oxygen, which scatter the sunlight in all directions. This scattering effect is most pronounced at shorter wavelengths, such as blue and violet light, which are scattered more than red light. As a result, these blue and violet light waves are present in the sky, giving it its blue appearance.

The reason why the sky appears blue under normal conditions is due to a combination of factors:

1. The atmosphere: The Earth's atmosphere scatters sunlight in all directions, with more scattering occurring at shorter wavelengths (blue and violet light).
2. Molecular size: The size of the molecules in the atmosphere is large enough to cause significant scattering of light.
3. Rayleigh's Law: The amount of scattering increases with the fourth power of the frequency of the light. This means t

### Embeddings

`nomic-embed-text` is ideal for embedding purpose, with shorter vector length (768) and faster speed

In [38]:
from time import time

In [40]:
txt = 'They sky is blue because of rayleigh scattering' * 10
for model_name in ['llama2','mistral','phi','yi', 'nomic-embed-text']:
    ts_start = time()
    e = ollama.embeddings(model=model_name, prompt=txt)['embedding']
    ts_stop = time()
    print(f"\nmodel: {model_name}, vector_len: {len(e)}, time: {(ts_stop-ts_start):.6f} \n\t vector[:10]: {e[:10]}")


model: llama2, vector_len: 4096, time: 0.855648 
	 vector[:10]: [-0.35514751076698303, 1.3837722539901733, 2.1101126670837402, 0.13837991654872894, -0.580007791519165, 1.0465730428695679, -0.216385617852211, 2.9255359172821045, -0.9857653379440308, 0.1334904432296753]

model: mistral, vector_len: 4096, time: 1.020040 
	 vector[:10]: [10.240944862365723, 3.288734197616577, 1.8220665454864502, 0.3301834464073181, 1.09401273727417, 0.7838320732116699, -4.774590492248535, 1.495103120803833, -9.1984224319458, 3.691711902618408]

model: phi, vector_len: 2560, time: 0.552677 
	 vector[:10]: [-0.5906329154968262, -1.5150843858718872, -1.3364429473876953, -0.7162874937057495, -0.29198890924453735, 0.2359294593334198, -0.7661153078079224, -1.1838363409042358, 0.4001242518424988, -0.4153246581554413]

model: yi, vector_len: 4096, time: 0.912859 
	 vector[:10]: [0.5858268737792969, 0.5222147703170776, -2.678344249725342, 0.322439044713974, 0.5310282707214355, 0.7788995504379272, 1.766578674316406

## Custom Client

- `host`: The Ollama host to connect to
- `timeout`: The timeout for requests

### Sync

In [41]:
from ollama import Client
host_url = 'http://localhost:11434'
timeout_sec = 10

client = Client(host=host_url, timeout=timeout_sec)
response = client.chat(model='llama2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
answer = response['message']['content']
print(answer)


The sky appears blue because of a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths. This is known as Rayleigh scattering.

As a result of this scattering, the blue light is dispersed throughout the atmosphere, giving the sky its blue color. The red light, on the other hand, is able to travel longer distances without being scattered, so it appears more red than blue. This is why the sun appears yellow or orange during sunrise and sunset, when the light has traveled through a longer distance in the atmosphere.

Other factors can also affect the color of the sky, such as pollution, dust, and water vapor. However, Rayleigh scattering is the primary reason why the sky appears blue.


### Async

In [42]:
import asyncio
from ollama import AsyncClient

async def chat():
  message = {'role': 'user', 'content': 'Why is the sky blue?'}
  response = await AsyncClient().chat(model='llama2', messages=[message])
  answer = response['message']['content']
  print(answer)

asyncio.run(chat())

RuntimeError: asyncio.run() cannot be called from a running event loop

## LangChain Integration

In [43]:
from langchain_community.llms import Ollama

In [44]:
llm = Ollama(model="llama2")

In [46]:
resp = llm.invoke("Tell me a joke about bear")
print(resp)

Why did the bear go to the restaurant?

Because he heard the service was "paws-itively" good!


In [47]:
llm = Ollama(model="mistral")
llm("The first man on the summit of Mount Everest, the highest peak on Earth, was ...")

/home/gongai/anaconda3/envs/ollama/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' The first person to reach the summit of Mount Everest was Sir Edmund Hillary from New Zealand and Tenzing Norgay, a Sherpa from Nepal, on May 29, 1953. They accomplished this feat as part of a British expedition led by John Hunt. This event is celebrated globally and marked an important milestone in mountaineering history.'

In [48]:
llm = Ollama(model="llama2")
llm("The first man on the summit of Mount Everest, the highest peak on Earth, was ...")

"\nThe first man to reach the summit of Mount Everest was Sir Edmund Hillary from New Zealand, who achieved this feat along with his Sherpa guide Tenzing Norgay on May 29, 1953. They used the South Col route and reached the summit at 11:30 am local time after a grueling climb that took several days. Hillary's achievement was a historic moment that marked the first time anyone had reached the summit of the world's highest mountain."